In [1]:
# For data processing
import numpy as np
from math import sqrt
# For data processing and manipulation
import pandas as pd
import csv
 
# For date calculations
import datetime
import time
 
# For ploting data
import IPython
import IPython.display
 
import itertools
from itertools import cycle
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 
# For checking path
import os , gc
from os import path
import csv
import json
from IPython.display import clear_output 
 
from scipy.stats import hmean
 
from sklearn.metrics import mean_squared_error
import requests

import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [2]:
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
'''
Should move db code to other file
'''
# connect to MongoDB, change the << MONGODB URL >> to reflect your own connection string
client = MongoClient(port=27017)
db = client.test

test query

In [8]:
# Record the start time
start_time = time.time()

requestData = {
    'wx_phrase': 'Mostly Cloudy',
    'pressure': 1003.99}
#urlTest = DATABASE_URL + "/query/find/chicago/weather"
#response = requests.get(urlTest, json=requestData)
response = db.weather.sf.find(requestData)

#do something with res
#print(response)
for res in response:
    display(res)


# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.4f} seconds".format(execution_time))
"""
if response.ok:
    try:
        data = response.json()
        print(len(data))
    except json.JSONDecodeError as e:
        print('Error in decoding json: ' + e)
    else:  
        print("Error:", response.status_code)"""

Execution Time: 0.0914 seconds


'\nif response.ok:\n    try:\n        data = response.json()\n        print(len(data))\n    except json.JSONDecodeError as e:\n        print(\'Error in decoding json: \' + e)\n    else:  \n        print("Error:", response.status_code)'

In [5]:
del response

<h1> Columns </h1>

In [3]:
# Record the start time
start_time = time.time()

# Get a single document from the collection
sample_document = db.weather.sf.find_one()

# Extract field names from the document
field_names = list(sample_document.keys())

print("Field Names:", field_names)
print("Number of Fields:", len(field_names))

# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.2f} seconds".format(execution_time))


Field Names: ['_id', '', 'expire_time_gmt', 'valid_time_gmt', 'temp', 'wx_phrase', 'heat_index', 'rh', 'pressure', 'vis', 'wc', 'wdir', 'wspd', 'max_temp', 'min_temp', 'expire_date_time', 'valid_date_time']
Number of Fields: 17
Execution Time: 0.01 seconds


In [13]:
del sample_document, field_names

<h3><b> Phrase Types without Pipeline </h3></b>

In [44]:
# Record the start time
start_time = time.time()

requestData = {
 'field': 'wx_phrase'}

#urlTest = DATABASE_URL + "/query/unique/chicago/weather"
#x = requests.get(urlTest, json=requestData)
distinct_wx_phrases = db.weather.sf.distinct('wx_phrase')
# Print the distinct values
for phrase in distinct_wx_phrases:
    print(phrase)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))

Cloudy
Cloudy / Windy
Drizzle
Fair
Fair / Windy
Fog
Fog / Windy
Haze
Haze / Windy
Heavy Rain
Heavy Rain / Windy
Heavy T-Storm
Heavy T-Storm / Windy
Light Drizzle
Light Drizzle / Windy
Light Rain
Light Rain / Windy
Light Rain with Thunder
Mist
Mostly Cloudy
Mostly Cloudy / Windy
Partly Cloudy
Partly Cloudy / Windy
Patches of Fog
Patches of Fog / Windy
Rain
Rain / Windy
Smoke
Smoke / Windy
Squalls
Squalls / Windy
T-Storm
T-Storm / Windy
Thunder
Thunder / Windy
Thunder in the Vicinity
Execution Time: 0.10 seconds


In [20]:
del distinct_wx_phrases

<h3><b> 2.2.2 Phrase Types using Pipeline </h3></b>

In [20]:
# Record the start time
start_time = time.time()

phraseXCount = {}
#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
                "_id": "$wx_phrase",
                "Count" : {"$sum" : 1}
        }
    }
]
# Run the aggregation query
data_cursor = db.weather.sf.aggregate(pipeline)

# Convert cursor to list (if dataset is large, consider processing each document in a loop instead)
data = list(data_cursor)

# Initialize variables
phraseXCount = {}
tsum = sum(doc['Count'] for doc in data)
c = 0

# Process each document
for doc in data:
    print(doc)
    phraseXCount[doc['_id']] = doc['Count']
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total Unique Types =", c)
print("Total Cases =", tsum)
print("Execution Time: {:.2f} seconds".format(execution_time))

#0.081 seconds

{'_id': 'Thunder', 'Count': 10}
{'_id': 'Cloudy / Windy', 'Count': 1713}
{'_id': 'Fair', 'Count': 65223}
{'_id': 'Partly Cloudy / Windy', 'Count': 3790}
{'_id': 'Fog / Windy', 'Count': 5}
{'_id': 'T-Storm', 'Count': 12}
{'_id': 'Heavy T-Storm / Windy', 'Count': 5}
{'_id': 'Haze', 'Count': 461}
{'_id': 'Squalls', 'Count': 4}
{'_id': 'Squalls / Windy', 'Count': 2}
{'_id': 'Fair / Windy', 'Count': 8122}
{'_id': 'Rain', 'Count': 575}
{'_id': 'Light Rain with Thunder', 'Count': 8}
{'_id': 'T-Storm / Windy', 'Count': 7}
{'_id': None, 'Count': 11}
{'_id': 'Light Rain / Windy', 'Count': 360}
{'_id': 'Heavy Rain / Windy', 'Count': 47}
{'_id': 'Cloudy', 'Count': 31008}
{'_id': 'Fog', 'Count': 808}
{'_id': 'Partly Cloudy', 'Count': 24386}
{'_id': 'Thunder in the Vicinity', 'Count': 13}
{'_id': 'Mist', 'Count': 89}
{'_id': 'Light Drizzle / Windy', 'Count': 3}
{'_id': 'Thunder / Windy', 'Count': 1}
{'_id': 'Mostly Cloudy / Windy', 'Count': 3449}
{'_id': 'Patches of Fog / Windy', 'Count': 2}
{'_id':

<h3><b> 2.2.5 working Average Temprature</h3></b>

In [30]:
# Record the start time
start_time = time.time()

pipeline = [
    {
        "$group": {
            "_id": None,
            "avg_temperature": { "$avg": "$temp" }
        }
    }
]
# Run the aggregation query
data_cursor = db.weather.sf.aggregate(pipeline)

# Convert cursor to list (if dataset is large, consider processing each document in a loop instead)
data = list(data_cursor)

# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.4f} seconds".format(execution_time))
        

{'_id': None, 'avg_temperature': 14.218140921289981}
Execution Time: 0.0707 seconds


<h3><b>2.2.4 Max Temprature </h3></b>

In [56]:
# Record the start time
start_time = time.time()

#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
            "_id": None,
            "max_temperature": { "$max": "$temp" }
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.sf.aggregate(pipeline)

# Convert cursor to list (if dataset is large, consider processing each document in a loop instead)
data = list(data_cursor)

# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': None, 'max_temperature': 39.0}
Execution Time: 0.06 seconds


<h3><b> 2.2.3 working min Temprature </h3></b>

In [60]:
# Record the start time
start_time = time.time()

#rlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
            "_id": None,
            "min_temperature": { "$min": "$temp" }
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.sf.aggregate(pipeline)

# Process each document in the cursor
for doc in data_cursor:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'_id': None, 'min_temperature': -12.0}
Execution Time: 0.07 seconds


<h3><b>working max temp per year </h3></b>

In [11]:
# Record the start time
start_time = time.time()

#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"

# Aggregation pipeline
pipeline = [
    {
        "$addFields": {
            "expire_date_time": {
                "$dateFromString": {
                    "dateString": "$expire_date_time",
                    "format": "%d/%m/%Y %H:%M:%S"  # Adjusted date format here
                }
            }
        }
    },
    {
        "$addFields": {
            "year": {"$year": "$expire_date_time"}
        }
    },
    {
        "$group": {
            "_id": "$year",
            "max_temperature": {"$max": "$temp"}
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.sf.aggregate(pipeline)

# Stream process each document in the cursor
for doc in data_cursor:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': 2004, 'max_temperature': 34.0}
{'_id': 2005, 'max_temperature': 31.0}
{'_id': 2006, 'max_temperature': 35.0}
{'_id': 2007, 'max_temperature': 31.0}
{'_id': 2008, 'max_temperature': 34.0}
{'_id': 2009, 'max_temperature': 33.0}
{'_id': 2010, 'max_temperature': 37.0}
{'_id': 2011, 'max_temperature': 32.0}
{'_id': 2012, 'max_temperature': 33.0}
{'_id': 2013, 'max_temperature': 31.0}
{'_id': 2014, 'max_temperature': 34.0}
{'_id': 2015, 'max_temperature': 33.0}
{'_id': 2016, 'max_temperature': 34.0}
{'_id': 2017, 'max_temperature': 38.0}
{'_id': 2018, 'max_temperature': 34.0}
{'_id': 2019, 'max_temperature': 36.0}
{'_id': 2020, 'max_temperature': 39.0}
{'_id': 2021, 'max_temperature': 32.0}
{'_id': 2022, 'max_temperature': 37.0}
Execution Time: 0.41 seconds


<h1><b> Query 2.1.1 Count Rainy Days </h1></b>

In [14]:
# Record the start time
start_time = time.time()
import re
match_word = "Rain"
# Aggregation pipeline to match wx_phrase with 'Rain' (case insensitive)
pipeline = [
    {
        "$match": {
            "wx_phrase": {"$regex": match_word, "$options": "i"}
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.sf.aggregate(pipeline)

# Initialize count
c = 0
rainXcount = {}

# Stream process each document in the cursor
for doc in data_cursor:
    rainXcount[doc['_id']] = doc['expire_date_time']
    print(f"On date : {doc['expire_date_time']} , {doc['wx_phrase']} was observed")
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total Cases =", c)
print("Execution Time: {:.2f} seconds".format(execution_time))

#.105 sconds

On date : 26/03/2014 03:00:00 , Rain was observed
On date : 26/03/2014 05:00:00 , Light Rain was observed
On date : 26/03/2014 20:00:00 , Light Rain was observed
On date : 26/03/2014 21:00:00 , Light Rain was observed
On date : 26/03/2014 22:00:00 , Light Rain was observed
On date : 26/03/2014 23:00:00 , Light Rain was observed
On date : 27/03/2014 02:00:00 , Light Rain was observed
On date : 27/03/2014 03:00:00 , Heavy Rain was observed
On date : 27/03/2014 04:00:00 , Light Rain was observed
On date : 27/03/2014 23:00:00 , Light Rain was observed
On date : 28/03/2014 17:00:00 , Light Rain was observed
On date : 29/03/2014 21:00:00 , Light Rain was observed
On date : 29/03/2014 22:00:00 , Light Rain was observed
On date : 29/03/2014 23:00:00 , Light Rain was observed
On date : 30/03/2014 00:00:00 , Light Rain was observed
On date : 30/03/2014 01:00:00 , Light Rain was observed
On date : 30/03/2014 02:00:00 , Light Rain was observed
On date : 30/03/2014 02:00:00 , Rain was observed
On d

<h1><b>2.2.1 Wind Direction Distribution </h1></b>


In [33]:
# Record the start time
start_time = time.time()
# Aggregation pipeline for wind direction count
pipeline = [
    {
        "$group": {
            "_id": "$wdir",
            "Intensity": {"$sum": 1}
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.sf.aggregate(pipeline)

# Initialize variables
wdirXCount = {}
tsum = 0
c = 0

# Stream process each document in the cursor
for doc in data_cursor:
    print(doc)
    wdirXCount[doc['_id']] = doc['Intensity']
    tsum += doc['Intensity']
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total directions =", c)
print("Total Cases =", tsum)
print("Execution Time: {:.3f} seconds".format(execution_time))

{'_id': None, 'Intensity': 23110}
{'_id': 10.0, 'Intensity': 2204}
{'_id': 19.0, 'Intensity': 1}
{'_id': 20.0, 'Intensity': 2313}
{'_id': 30.0, 'Intensity': 2302}
{'_id': 40.0, 'Intensity': 2023}
{'_id': 50.0, 'Intensity': 1745}
{'_id': 60.0, 'Intensity': 1680}
{'_id': 70.0, 'Intensity': 1535}
{'_id': 80.0, 'Intensity': 1441}
{'_id': 90.0, 'Intensity': 1256}
{'_id': 100.0, 'Intensity': 1361}
{'_id': 110.0, 'Intensity': 1269}
{'_id': 120.0, 'Intensity': 1767}
{'_id': 130.0, 'Intensity': 2258}
{'_id': 140.0, 'Intensity': 2241}
{'_id': 150.0, 'Intensity': 2220}
{'_id': 160.0, 'Intensity': 2400}
{'_id': 170.0, 'Intensity': 2664}
{'_id': 180.0, 'Intensity': 2619}
{'_id': 190.0, 'Intensity': 2710}
{'_id': 200.0, 'Intensity': 2821}
{'_id': 210.0, 'Intensity': 2693}
{'_id': 220.0, 'Intensity': 2738}
{'_id': 230.0, 'Intensity': 2998}
{'_id': 240.0, 'Intensity': 3650}
{'_id': 250.0, 'Intensity': 5648}
{'_id': 260.0, 'Intensity': 11597}
{'_id': 270.0, 'Intensity': 18574}
{'_id': 280.0, 'Intensity

<h1><b> Query 2.1.2 In A Range Check</h1></b>


In [17]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$match": {
            "$and": [
                {"pressure": {"$gt": 1000.00}},
                {"pressure": {"$lt": 1001.00}}
            ]
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.sf.aggregate(pipeline)

# Initialize count
c = 0

# Stream process each document in the cursor
for doc in data_cursor:
    print(doc)
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total Cases =", c)
print("Execution Time: {:.2f} seconds".format(execution_time))
#{"pressure": {"$gt": 1000.00, "$lt": 1001.00}}
#.077 seconds

{'_id': ObjectId('65b74692cf903dd4f83d91c9'), '': 7561, 'expire_time_gmt': 1098262560, 'valid_time_gmt': 1098255360, 'temp': 13.0, 'wx_phrase': 'Cloudy', 'heat_index': 13.0, 'rh': 89.0, 'pressure': 1000.54, 'vis': 6.0, 'wc': 13.0, 'wdir': 270.0, 'wspd': 15.0, 'expire_date_time': '20/10/2004 15:00:00', 'valid_date_time': '20/10/2004 13:00:00'}
{'_id': ObjectId('65b74692cf903dd4f83d91cc'), '': 7564, 'expire_time_gmt': 1098273360, 'valid_time_gmt': 1098266160, 'temp': 13.0, 'wx_phrase': 'Mostly Cloudy', 'heat_index': 13.0, 'rh': 80.0, 'pressure': 1000.24, 'vis': 16.0, 'wc': 13.0, 'wdir': 150.0, 'wspd': 20.0, 'expire_date_time': '20/10/2004 18:00:00', 'valid_date_time': '20/10/2004 16:00:00'}
{'_id': ObjectId('65b74692cf903dd4f83d91cd'), '': 7565, 'expire_time_gmt': 1098274740, 'valid_time_gmt': 1098267540, 'temp': 11.0, 'wx_phrase': 'Cloudy', 'heat_index': 11.0, 'rh': 88.0, 'pressure': 1000.66, 'vis': 4.0, 'wc': 11.0, 'wdir': 240.0, 'wspd': 19.0, 'expire_date_time': '20/10/2004 18:00:00',

<h1> <b> Complex Queries </b> </h1>

<h3> Monthly Average Temperature, Wind Speed with Condition Counts </h3>

In [4]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$match": {
            "pressure": { "$gte": 990, "$lte": 1020 }
        }
    },
    {
        "$project": {
            "month": { "$month": { "$dateFromString": { "dateString": "$valid_date_time", "format": "%d/%m/%Y %H:%M:%S" } } },
            "temp": 1,
            "wspd": 1,
            "wx_phrase": 1
        }
    },
    {
        "$group": {
            "_id": "$month",
            "avg_temp": { "$avg": "$temp" },
            "avg_wspd": { "$avg": "$wspd" },
            "distinct_wx_phrases": { "$addToSet": "$wx_phrase" }
        }
    },
    {
        "$project": {
            "month": "$_id",
            "avg_temp": 1,
            "avg_wspd": 1,
            "distinct_conditions": { "$size": "$distinct_wx_phrases" }
        }
    },
    {
        "$match": {
            "avg_wspd": { "$gt": 10 }
        }
    },
    {
        "$sort": {
            "month": 1
        }
    }
]

# Run the aggregation query
data = db.weather.sf.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.003 seconds

,_id,avg_temp,avg_wspd,month,distinct_conditions
0,1,11.069614,15.452476,1,26
1,2,11.888727,17.371890,2,23
2,3,12.654699,18.028635,3,23
3,4,13.524111,20.552475,4,22
4,5,14.575238,22.497615,5,20
5,6,16.224868,22.139140,6,12
6,7,16.709088,20.447994,7,14
7,8,17.143947,19.410616,8,20
8,9,17.499485,18.444971,9,20
9,10,16.395249,15.844594,10,19


Execution Time: 0.57 seconds


<h3> Daily Data Analysis </h3>

In [5]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline =[
    {
        "$addFields": {
            "day": {
                "$dateToString": {
                    "format": "%Y-%m-%d",
                    "date": {
                        "$dateFromString": {
                            "dateString": "$valid_date_time",
                            "format": "%d/%m/%Y %H:%M:%S"
                        }
                    }
                }
            }
        }
    },
    {
        "$group": {
            "_id": "$day",
            "max_max_temp": { "$max": "$max_temp" },
            "min_min_temp": { "$min": "$min_temp" },
            "min_vis": { "$min": "$vis" }
        }
    },
    {
        "$addFields": {
            "temp_variation": { "$subtract": ["$max_max_temp", "$min_min_temp"] }
        }
    },
    {
        "$match": {
            "temp_variation": { "$gt": 15 },
            "min_vis": { "$lt": 5 }
        }
    },
    {
        "$sort": { "temp_variation": -1 }
    },
    {
        "$project": {
            "_id": 0,
            "day": "$_id",
            "temp_variation": 1,
            "min_visibility": "$min_vis"
        }
    }
]

# Run the aggregation query
data = db.weather.sf.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#.123 seconds

,temp_variation,day,min_visibility
0,48.0,2016-12-16,2.0
1,39.0,2020-12-02,1.0
2,29.0,2006-03-06,4.0
3,25.0,2017-01-21,2.0
4,21.0,2020-09-30,1.0
5,17.0,2013-03-14,0.4
6,17.0,2021-01-22,4.0
7,17.0,2020-10-03,4.0
8,17.0,2005-12-19,2.0


Execution Time: 0.43 seconds


<h3> Extreme weather days <h3>

In [8]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline =[
    {
        "$addFields": {
            "day": {
                "$dateFromString": {
                    "dateString": {
                        "$substr": ["$valid_date_time", 0, 10]  # Extracting only the date part
                    },
                    "format": "%d/%m/%Y"
                }
            }
        }
    },
    {
        "$group": {
            "_id": "$day",
            "max_temp": { "$max": "$temp" },
            "max_wspd": { "$max": "$wspd" },
            "min_vis": { "$min": "$vis" }
        }
    },
    {
        "$match": {
            "$or": [
                { "max_temp": { "$gt": 30 } },
                { "max_wspd": { "$gt": 50 } },
                { "min_vis": { "$lt": 2 } }
            ]
        }
    },
    {
        "$sort": { "_id": 1 }
    },
    {
        "$project": {
            "_id": 0,
            "day": "$_id",
            "max_temp": 1,
            "max_wspd": 1,
            "min_vis": 1
        }
    }
]



# Run the aggregation query
data = db.weather.sf.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#.122 seconds

,max_temp,max_wspd,min_vis,day
0,12.0,70.0,1.0,2004-01-01
1,13.0,61.0,1.0,2004-01-02
2,16.0,13.0,0.4,2004-01-12
3,12.0,17.0,0.4,2004-01-13
4,18.0,72.0,2.0,2004-02-18
...,...,...,...,...
699,21.0,33.0,1.0,2022-10-08
700,19.0,52.0,14.0,2022-10-23
701,20.0,33.0,0.8,2022-10-30
702,14.0,44.0,1.0,2022-11-02


Execution Time: 0.37 seconds


<h3> phrase wise temprature pressure <h3>

In [16]:
from datetime import datetime

# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "parsed_date": {
                "$dateFromString": {
                    "dateString": "$valid_date_time",
                    "format": "%d/%m/%Y %H:%M:%S"
                }
            }
        }
    },
    {
        "$match": {
            "parsed_date": {
                "$gte": datetime(2006, 6, 1),
                "$lt": datetime(2007, 6, 1)
            }
        }
    },
    {
        "$group": {
            "_id": "$wx_phrase",
            "max_temp": {"$max": "$temp"},
            "max_pressure": {"$max": "$pressure"}
        }
    },
    {
        "$sort": {"_id": 1}
    },
    {
        "$project": {
            "_id": 0,
            "wx_phrase": "$_id",
            "max_temp": 1,
            "max_pressure": 1
        }
    }
]

# Run the aggregation query
data = db.weather.sf.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#.125 seconds

,max_temp,max_pressure,wx_phrase
0,21.0,NaN,None
1,22.0,1030.79,Cloudy
2,19.0,1021.94,Cloudy / Windy
3,35.0,1035.43,Fair
4,29.0,1032.03,Fair / Windy
5,25.0,1033.73,Mostly Cloudy
6,23.0,1030.79,Mostly Cloudy / Windy
7,29.0,1033.23,Partly Cloudy
8,26.0,1026.24,Partly Cloudy / Windy


Execution Time: 0.37 seconds


<h1><b> addition </b></h1>

<h1><b> subtraction </b></h1>

<h1><b> comparing </b></h1>

<h3> Query 2.1.3 Identifying records where the wind chill is less than the actual temperature and the wind speed is above a certain level</h3>

In [4]:
from datetime import datetime

# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline  = [
    {
        "$addFields": {
            "wind_chill_gap": {"$subtract": ["$temp", "$wc"]},
            "temperature_comparison": {
                "$cond": {
                    "if": {"$lt": ["$wc", "$temp"]},
                    "then": "Wind Chill Lower",
                    "else": "Temperature Lower or Equal"
                }
            },
            "wind_speed_status": {
                "$cond": {
                    "if": {"$gt": ["$wspd", 25]},
                    "then": "High Wind Speed",
                    "else": "Normal Wind Speed"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "valid_time_gmt": 1,
            "temp": 1,
            "wc": 1,
            "wspd": 1,
            "wind_chill_gap": 1,
            "temperature_comparison": 1,
            "wind_speed_status": 1,
            "wx_phrase": 1,
            "valid_date_time": 1
        }
    }
]

# Run the aggregation query
data = db.weather.sf.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#.137 seconds

#{ "valid_date_time": 1,"wx_phrase": 1,"wspd": 1,"wc": 1,"temp": 1,"valid_time_gmt": 1,"Unnamed: 0": 1,"_id": 0,"wind_chill_gap": { "$subtract": ["$temp", "$wc"] }, "temperature_comparison": { "$cond": { "if": { "$lt": ["$wc", "$temp"] }, "then": "Wind Chill Lower", "else": "Temperature Lower or Equal" } }, "wind_speed_status": { "$cond": { "if": { "$gt": ["$wspd", 25] }, "then": "High Wind Speed", "else": "Normal Wind Speed" } } }
#.210 seconds

,valid_time_gmt,temp,wx_phrase,wc,wspd,valid_date_time,wind_chill_gap,temperature_comparison,wind_speed_status
0,1072943760,12.0,Cloudy,12.0,22.0,01/01/2004 14:00:00,0.0,Temperature Lower or Equal,Normal Wind Speed
1,1072947360,11.0,Cloudy,11.0,22.0,01/01/2004 15:00:00,0.0,Temperature Lower or Equal,Normal Wind Speed
2,1072950960,11.0,Cloudy,11.0,20.0,01/01/2004 16:00:00,0.0,Temperature Lower or Equal,Normal Wind Speed
3,1072951440,11.0,Cloudy,11.0,24.0,01/01/2004 16:00:00,0.0,Temperature Lower or Equal,Normal Wind Speed
4,1072954560,11.0,Cloudy / Windy,11.0,33.0,01/01/2004 17:00:00,0.0,Temperature Lower or Equal,High Wind Speed
...,...,...,...,...,...,...,...,...,...
181867,1672463580,16.0,Light Rain,16.0,26.0,31/12/2022 11:00:00,0.0,Temperature Lower or Equal,High Wind Speed
181868,1672466160,16.0,Light Rain,16.0,30.0,31/12/2022 12:00:00,0.0,Temperature Lower or Equal,High Wind Speed
181869,1672466880,16.0,Light Rain,16.0,30.0,31/12/2022 12:00:00,0.0,Temperature Lower or Equal,High Wind Speed
181870,1672469760,15.0,Rain,15.0,19.0,31/12/2022 13:00:00,0.0,Temperature Lower or Equal,Normal Wind Speed


Execution Time: 0.96 seconds


<h3> Query 2.1.4 Checking for temperature variations and categorizing weather conditions based on wind speed and temperature: </h3>

In [5]:
from datetime import datetime

# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "temp_range": {"$subtract": ["$max_temp", "$min_temp"]},
            "adjusted_wspd": {"$add": ["$wspd", 10]},
            "weather_condition": {
                "$switch": {
                    "branches": [
                        {"case": {"$gt": ["$wspd", 20]}, "then": "Windy"},
                        {"case": {"$lt": ["$temp", 32]}, "then": "Freezing"}
                    ],
                    "default": "Mild"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "valid_time_gmt": 1,
            "temp_range": 1,
            "adjusted_wspd": 1,
            "weather_condition": 1,
            "wx_phrase": 1,
            "valid_date_time": 1
        }
    }
]


# Run the aggregation query
data = db.weather.sf.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#.131 seconds

#{"valid_time_gmt": 1,"_id": 0,"temp_range": {"$subtract": ["$max_temp", "$min_temp"]}, "adjusted_wspd": {"$add": ["$wspd", 10]}, "weather_condition": {"$cond": [{"$gt": ["$wspd", 20]}, "Windy", {"$cond": [{"$lt": ["$temp", 32]}, "Freezing", "Mild"]}]}, "wx_phrase": 1, "valid_date_time": 1}
#.207 seconds

,valid_time_gmt,wx_phrase,valid_date_time,temp_range,adjusted_wspd,weather_condition
0,1072943760,Cloudy,01/01/2004 14:00:00,5.0,32.0,Windy
1,1072947360,Cloudy,01/01/2004 15:00:00,NaN,32.0,Windy
2,1072950960,Cloudy,01/01/2004 16:00:00,NaN,30.0,Freezing
3,1072951440,Cloudy,01/01/2004 16:00:00,NaN,34.0,Windy
4,1072954560,Cloudy / Windy,01/01/2004 17:00:00,NaN,43.0,Windy
...,...,...,...,...,...,...
181867,1672463580,Light Rain,31/12/2022 11:00:00,NaN,36.0,Windy
181868,1672466160,Light Rain,31/12/2022 12:00:00,NaN,40.0,Windy
181869,1672466880,Light Rain,31/12/2022 12:00:00,NaN,40.0,Windy
181870,1672469760,Rain,31/12/2022 13:00:00,NaN,29.0,Freezing


Execution Time: 0.80 seconds


<h3> Query 2.1.5 Comparing day and night temperatures and visibility for weather predictions </h3>

In [18]:
from datetime import datetime

# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "combined_temperature": {"$add": ["$max_temp", "$min_temp"]},
            "half_visibility": {"$divide": ["$vis", 2]},
            "weather_prediction": {
                "$switch": {
                    "branches": [
                        {
                            "case": {"$and": [
                                {"$gt": ["$max_temp", 75]},
                                {"$lt": ["$min_temp", 50]}
                            ]},
                            "then": "Wide Temperature Range"
                        },
                        {"case": {"$lt": ["$vis", 5]}, "then": "Foggy"}
                    ],
                    "default": "Clear"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "valid_time_gmt": 1,
            "combined_temperature": 1,
            "half_visibility": 1,
            "weather_prediction": 1,
            "wx_phrase": 1,
            "valid_date_time": 1
        }
    }
]


# Run the aggregation query
data = db.weather.sf.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#.125 seconds

#{"valid_time_gmt": 1,"Unnamed: 0": 1,"_id": 0,"combined_temperature": {"$add": ["$max_temp", "$min_temp"]}, "half_visibility": {"$divide": ["$vis", 2]}, "weather_prediction": {"$switch": {"branches": [{"case": {"$and": [{"$gt": ["$max_temp", 75]}, {"$lt": ["$min_temp", 50]}]},"then": "Wide Temperature Range"}, {"case": {"$lt": ["$vis", 5]},"then": "Foggy"}], "default": "Clear"}}, "wx_phrase": 1, "valid_date_time": 1}
#.209 seconds

,valid_time_gmt,wx_phrase,valid_date_time,combined_temperature,half_visibility,weather_prediction
0,1072943760,Cloudy,01/01/2004 14:00:00,19.0,8.0,Clear
1,1072947360,Cloudy,01/01/2004 15:00:00,NaN,6.0,Clear
2,1072950960,Cloudy,01/01/2004 16:00:00,NaN,4.0,Clear
3,1072951440,Cloudy,01/01/2004 16:00:00,NaN,3.0,Clear
4,1072954560,Cloudy / Windy,01/01/2004 17:00:00,NaN,4.0,Clear
...,...,...,...,...,...,...
181867,1672463580,Light Rain,31/12/2022 11:00:00,NaN,2.0,Foggy
181868,1672466160,Light Rain,31/12/2022 12:00:00,NaN,4.0,Clear
181869,1672466880,Light Rain,31/12/2022 12:00:00,NaN,2.0,Foggy
181870,1672469760,Rain,31/12/2022 13:00:00,NaN,2.0,Foggy


Execution Time: 0.82 seconds
